<a href="https://colab.research.google.com/github/Zaryn-Ooi/Malaysia-Pay-Gap-/blob/main/MalaysianPayGap_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Do Malaysians like their job ?
Aim: To identify whether most malaysians are satisfied with their work or not with the use of NLP.

Data Source: Malaysian Pay Gap Instagram Account

Data Content: Text within the images of a post


In [1]:
# Import packages
import pandas as pd
import numpy as np


In [3]:
# Load data
from google.colab import files
uploaded = files.upload()

Saving posts_text.csv to posts_text.csv


In [4]:
df = pd.read_csv('posts_text.csv')
df.head()

,hashtags,captions,likes,comments,image_ids,image_paths,image_text,Unnamed: 7
0,['#malaysianpaygap'],NaN,205,13,CaTj7N9P64r,/Users/aishapeng/Python/MalaysianPayGap-master...,Senior Sales Engineer in MNC\ncompany\n\n30\n\...,610
1,['#malaysianpaygap'],NaN,154,2,CaTiOpXvF3A,/Users/aishapeng/Python/MalaysianPayGap-master...,Marketing & PR\n27\n\nFemale\nChinese\n\n4 yea...,323
2,['#malaysianpaygap'],Congratulations on your first job out of uni! ...,149,14,CaUQxjsvuZk,/Users/aishapeng/Python/MalaysianPayGap-master...,Social media executive\n26\n\nFemale\n\nMalay\...,401
3,"['#malaysianpaygap', '#mpgcreative']",Passion-driven career often met with not-so-pa...,674,20,CaY3AMMPCvC,/Users/aishapeng/Python/MalaysianPayGap-master...,Animator\n\nin game/ tv series industry\n26 ch...,158
4,"['#malaysianpaygap', '#mpgcreative']",Passion-driven career often met with not-so-pa...,674,20,CaY3AMMPCvC,/Users/aishapeng/Python/MalaysianPayGap-master...,Started out with RM 2000 as a\nfresh grad.\n\n...,569


In [5]:
len(df)

218

In [7]:
# Detect and remove NaN values 
df.isnull().sum()

hashtags        0
captions       20
likes           0
comments        0
image_ids       0
image_paths     0
image_text      0
Unnamed: 7      0
dtype: int64

We're only looking if there is null values in the image_text. Since there is no null values in the image_text column, we do not need to remove anything. 

In [8]:
# Detect duplicated rows
df.duplicated().sum()

0

In [9]:
df.columns

Index(['hashtags', 'captions', 'likes', 'comments', 'image_ids', 'image_paths',
       'image_text', 'Unnamed: 7'],
      dtype='object')

## Data Cleaning

In [10]:
# Find the number of unique image id
num_of_image = len(df['image_ids'].unique())
num_of_image

156

In [11]:
# Concatenate rows with same image id, and make image id as index
df = df.groupby('image_ids',as_index=True).agg(lambda x: x.tolist())

In [12]:
len(df)

156

We realised that there are 218 rows of data, however there are only 156 unique image id. This means that some of the posts(image id) have multiple images. Hence, we combined rows with the same image id. Now, there is no more duplicated image id. 

In [13]:
# Combine multiple lists into one list
df['image_text'] = df['image_text'].apply(lambda image_text: " ".join(image_text))

In [14]:
# Remove every last word of the image_text 
df['image_text']= df['image_text'].apply(lambda image_text: " ".join(image_text.split()[:-2]))

## Named Entity Recognition (NER)


In [ ]:
#import spacy
#nlp = spacy.load('en_core_web_sm')

In [ ]:
# Write a function to display basic entity info:
# def show_ents(df):
#    if df.ents:  # if it has named entities
#        for ent in df.ents:
#            print(ent.text+' - '+ent.label_+' - '+str(spacy.explain(ent.label_)))
#    else:
#        print('No named entities found.')

## Sentiment Analysis 

In [ ]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
# Import SentimentIntensityAnalyzer and create an sid object
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

In [ ]:
# Use sid to append a comp_score to the dataset
df['scores'] = df['image_text'].apply(lambda image_text:sid.polarity_scores(image_text))

df['compound'] = df['scores'].apply(lambda score_dict: score_dict['compound'])

df['comp_score'] = df['compound'].apply(lambda c: 'satisfied' if c>= 0.5 else 'neutral' if c>=0.1 else 'dissatisfied')

df.head()

,hashtags,captions,likes,comments,image_paths,image_text,scores,compound,comp_score
image_ids,,,,,,,,,
Ca0ueFAPokI,"[['#malaysianpaygap', '#mpghealthcare'], ['#ma...",[Our society is filled with so many different ...,"[1547, 1547]","[17, 17]",[/Users/aishapeng/Python/MalaysianPayGap-maste...,Behaviour Therapist (ABA Therapy) Workng w chi...,"{'neg': 0.061, 'neu': 0.795, 'pos': 0.144, 'co...",0.9304,satisfied
CaRp-1uPh8l,[['#malaysianpaygap']],[nan],[1109],[2],[/Users/aishapeng/Python/MalaysianPayGap-maste...,MALAYSIA'S MEDIAN SALARY LW 19 bY a bY BELOW T...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,dissatisfied
CaT5MguPpDI,[['#malaysianpaygap']],[Earning only stable currencies is on everyone...,[244],[18],[/Users/aishapeng/Python/MalaysianPayGap-maste...,Director 32 Male Chinese 9 years Bachelors 5-1...,"{'neg': 0.071, 'neu': 0.78, 'pos': 0.149, 'com...",0.9168,satisfied
CaT6d2Yve5X,[['#malaysianpaygap']],[The many forms of wage theft in local work sc...,[327],[10],[/Users/aishapeng/Python/MalaysianPayGap-maste...,IT (Healthcare / Cybersecurity) 45 Malay RM8k ...,"{'neg': 0.064, 'neu': 0.879, 'pos': 0.057, 'co...",0.0516,dissatisfied
CaTg8Ijvcsw,[['#malaysianpaygap']],[nan],[191],[4],[/Users/aishapeng/Python/MalaysianPayGap-maste...,PR/Marketing in NGO sector 28 Female Malay 6 y...,"{'neg': 0.053, 'neu': 0.806, 'pos': 0.141, 'co...",0.7842,satisfied


In [ ]:
df['comp_score'].value_counts()

satisfied       81
dissatisfied    58
neutral         17
Name: comp_score, dtype: int64

In [ ]:
# display percentage 
df['comp_score'].value_counts(normalize = True)

satisfied       0.519231
dissatisfied    0.371795
neutral         0.108974
Name: comp_score, dtype: float64

Based on the sentiment analysis , it shows that 50% of Malaysians are satisfied with their work. However, we need to further explore to see if this is true. 

In [ ]:
# display how satisfied jobs looks like
dissatisfied = df.loc[df['comp_score'] == 'dissatisfied']
dissatisfied.head()

,hashtags,captions,likes,comments,image_paths,image_text,scores,compound,comp_score
image_ids,,,,,,,,,
CaRp-1uPh8l,[['#malaysianpaygap']],[nan],[1109],[2],[/Users/aishapeng/Python/MalaysianPayGap-maste...,MALAYSIA'S MEDIAN SALARY LW 19 bY a bY BELOW T...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,dissatisfied
CaT6d2Yve5X,[['#malaysianpaygap']],[The many forms of wage theft in local work sc...,[327],[10],[/Users/aishapeng/Python/MalaysianPayGap-maste...,IT (Healthcare / Cybersecurity) 45 Malay RM8k ...,"{'neg': 0.064, 'neu': 0.879, 'pos': 0.057, 'co...",0.0516,dissatisfied
CaTh3SgvnI5,[['#malaysianpaygap']],[nan],[118],[17],[/Users/aishapeng/Python/MalaysianPayGap-maste...,Project Engineer in Food manufacturing Industr...,"{'neg': 0.148, 'neu': 0.852, 'pos': 0.0, 'comp...",-0.7736,dissatisfied
CaThAfwP7Nc,[['#malaysianpaygap']],[nan],[240],[20],[/Users/aishapeng/Python/MalaysianPayGap-maste...,Senior Barista in F&B 25 2 years experience 2....,"{'neg': 0.134, 'neu': 0.783, 'pos': 0.083, 'co...",-0.2075,dissatisfied
CaThuNHvOiu,[['#malaysianpaygap']],[nan],[61],[2],[/Users/aishapeng/Python/MalaysianPayGap-maste...,Senior Finance Logistics 28 Indian 5 years Rm4...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,dissatisfied


In [ ]:
neutral = df[df['comp_score'] == 'neutral']
# neutral

## Latent Semantic Analysis (LSA)

### Stemming

In [15]:
# perform stemming to reduce words to their root
from nltk.stem.snowball import SnowballStemmer

# The Snowball Stemmer requires that you pass a language parameter
s_stemmer = SnowballStemmer(language='english')

In [16]:
def stemmed_text(imagetext):
    return ' '.join([s_stemmer.stem(word) for word in imagetext.split()])

df['image_text_lem'] = df['image_text'].apply(stemmed_text)
print(df['image_text_lem'][:5]) 
df['image_text'][:5]

image_ids
Ca0ueFAPokI    behaviour therapist (aba therapy) workng w chi...
CaRp-1uPh8l    malaysia median salari lw 19 by a by below the...
CaT5MguPpDI    director 32 male chines 9 year bachelor 5-10k ...
CaT6d2Yve5X    it (healthcar / cybersecurity) 45 malay rm 60,...
CaTg8Ijvcsw    pr/market in ngo sector 28 femal malay 6 year ...
Name: image_text_lem, dtype: object


image_ids
Ca0ueFAPokI    Behaviour Therapist (ABA Therapy) Workng w chi...
CaRp-1uPh8l    MALAYSIA'S MEDIAN SALARY LW 19 bY a bY BELOW T...
CaT5MguPpDI    Director 32 Male Chinese 9 years Bachelors 5-1...
CaT6d2Yve5X    IT (Healthcare / Cybersecurity) 45 Malay RM 60...
CaTg8Ijvcsw    PR/Marketing in NGO sector 28 Female Malay 6 y...
Name: image_text, dtype: object

In [ ]:
# for imagetext in df['image_text'].values:
#   res = ""  
#   for word in imagetext.split():
#     res += s_stemmer.stem(word)
#     print(word+' ---> '+s_stemmer.stem(word))

### Document Term Matrix

In [30]:
# count term frequency and converts each document into a vector
from sklearn.feature_extraction.text import CountVectorizer

CV = CountVectorizer(max_df = 0.20, min_df = 2, stop_words='english') # remove stop words
bag_of_words = CV.fit_transform(df['image_text_lem']) 

In [31]:
# Transform the original text message to vector
vector = CV.transform(df['image_text_lem'])

In [32]:
vector.shape

(156, 1022)

min_df reduce the words from 3075 characters to 1085.
stemming has reduced the words from 1085 to 1062.
Remove last words and reduced max_df to 0.20 have reduced 1062 to 1022..

### Singular Value Decomposition (SVD)

In [33]:
bag_of_words

<156x1022 sparse matrix of type '<class 'numpy.int64'>'
	with 5103 stored elements in Compressed Sparse Row format>

In [34]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components = 2)
lsa = svd.fit_transform(bag_of_words)

In [35]:
# Constructing DataFrame from lsa
topic_encoded = pd.DataFrame(lsa, columns = ["topic_1", "topic_2"])
topic_encoded['text'] = df['image_text'].values
display(topic_encoded[["text", "topic_1", "topic_2"]])

,text,topic_1,topic_2
0,Behaviour Therapist (ABA Therapy) Workng w chi...,3.364208,0.343514
1,MALAYSIA'S MEDIAN SALARY LW 19 bY a bY BELOW T...,0.135947,-0.053438
2,Director 32 Male Chinese 9 years Bachelors 5-1...,2.165237,0.733433
3,IT (Healthcare / Cybersecurity) 45 Malay RM 60...,2.336879,-1.365057
4,PR/Marketing in NGO sector 28 Female Malay 6 y...,1.207181,0.355335
...,...,...,...
151,"Lead data science, manager, industry data anal...",1.554612,-0.027796
152,2 P ~— = rR a. EN a €,0.008018,0.012404
153,"Mobile Developer, Fintech MNC 22, East MY Bumi...",1.275746,0.102072
154,"strange for someone who's ""supposed"" to be hol...",2.134513,0.067230


In [ ]:
# extract first 2 words of image_text
# text = [' '.join(text.split()[:2]) for text in image_text]
# text[:5]

### Encoding

In [36]:
dictionary = CV.get_feature_names()
dictionary[:10]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['000', '10', '100', '10k', '10pm', '12', '12k', '14', '15', '15k']

### The Encoding Matrix

In [37]:
encoding_matrix = pd.DataFrame(svd.components_,
                               index=['topic_1', 'topic_2'],
                               columns=dictionary).T
encoding_matrix

,topic_1,topic_2
000,0.031436,-0.024965
10,0.050864,0.015893
100,0.005908,-0.001912
10k,0.007605,-0.000214
10pm,0.002520,-0.001951
...,...,...
ya,0.005097,-0.000965
years,0.027826,0.020558
young,0.026325,0.005498
younger,0.006324,0.001477


### Find the top words for each topic

In [38]:
encoding_matrix['abs_topic_1'] = np.abs(encoding_matrix['topic_1'])
encoding_matrix['abs_topic_2'] = np.abs(encoding_matrix['topic_2'])
top_20_topic1 = encoding_matrix.sort_values('abs_topic_1', ascending=False)
top_20_topic1.head(20)

,topic_1,topic_2,abs_topic_1,abs_topic_2
time,0.250149,-0.348853,0.250149,0.348853
veri,0.184425,0.195952,0.184425,0.195952
rm,0.175356,-0.199679,0.175356,0.199679
day,0.145593,-0.093089,0.145593,0.093089
need,0.145433,0.025384,0.145433,0.025384
know,0.144180,0.057089,0.144180,0.057089
onli,0.135349,0.017134,0.135349,0.017134
5k,0.135180,-0.038523,0.135180,0.038523
basic,0.124083,-0.052454,0.124083,0.052454
increment,0.120838,-0.003670,0.120838,0.003670


We can see that most of the top 20 words for **Topic 1** are more close to describing satisfied work, such as "better", "increment.Therefore, **Topic 1** refers to 'satisfied'.

In [39]:
top_20_topic2 = encoding_matrix.sort_values('abs_topic_2', ascending=False)
top_20_topic2.head(20)

,topic_1,topic_2,abs_topic_1,abs_topic_2
lawyer,0.087621,0.478234,0.087621,0.478234
time,0.250149,-0.348853,0.250149,0.348853
firm,0.068533,0.232817,0.068533,0.232817
law,0.050257,0.221758,0.050257,0.221758
rm,0.175356,-0.199679,0.175356,0.199679
veri,0.184425,0.195952,0.184425,0.195952
underpaid,0.079781,0.143078,0.079781,0.143078
especi,0.072281,0.140117,0.072281,0.140117
client,0.069323,0.114884,0.069323,0.114884
offer,0.095396,-0.113290,0.095396,0.113290


We can see that most of the top 20 words for **Topic 2** are more close to describing dissatisfied work, such as "underpaid", "bad". Therefore, **Topic 2** refers to **'dissatisfied'**.

# Label topic 

In [44]:
topic_encoded = topic_encoded.rename(columns={'topic_1': 'satisfied', 'topic_2': 'dissatisfied'})
display(topic_encoded[["text", "satisfied", "dissatisfied"]])

,text,satisfied,dissatisfied
0,Behaviour Therapist (ABA Therapy) Workng w chi...,3.364208,0.343514
1,MALAYSIA'S MEDIAN SALARY LW 19 bY a bY BELOW T...,0.135947,-0.053438
2,Director 32 Male Chinese 9 years Bachelors 5-1...,2.165237,0.733433
3,IT (Healthcare / Cybersecurity) 45 Malay RM 60...,2.336879,-1.365057
4,PR/Marketing in NGO sector 28 Female Malay 6 y...,1.207181,0.355335
...,...,...,...
151,"Lead data science, manager, industry data anal...",1.554612,-0.027796
152,2 P ~— = rR a. EN a €,0.008018,0.012404
153,"Mobile Developer, Fintech MNC 22, East MY Bumi...",1.275746,0.102072
154,"strange for someone who's ""supposed"" to be hol...",2.134513,0.067230


## Topic Modelling

In [49]:
topic_encoded["dissatisfied"] = np.abs(topic_encoded["dissatisfied"])
topic_encoded["satisfied"] = np.abs(topic_encoded["satisfied"])

In [50]:
topic_encoded["label"] = topic_encoded.apply(lambda row: 'satisfied' if row["satisfied"] >= row["dissatisfied"] else 'dissatisfied', axis=1)
display(topic_encoded[["text", "satisfied", "dissatisfied", "label"]])

,text,satisfied,dissatisfied,label
0,Behaviour Therapist (ABA Therapy) Workng w chi...,3.364208,0.343514,satisfied
1,MALAYSIA'S MEDIAN SALARY LW 19 bY a bY BELOW T...,0.135947,0.053438,satisfied
2,Director 32 Male Chinese 9 years Bachelors 5-1...,2.165237,0.733433,satisfied
3,IT (Healthcare / Cybersecurity) 45 Malay RM 60...,2.336879,1.365057,satisfied
4,PR/Marketing in NGO sector 28 Female Malay 6 y...,1.207181,0.355335,satisfied
...,...,...,...,...
151,"Lead data science, manager, industry data anal...",1.554612,0.027796,satisfied
152,2 P ~— = rR a. EN a €,0.008018,0.012404,dissatisfied
153,"Mobile Developer, Fintech MNC 22, East MY Bumi...",1.275746,0.102072,satisfied
154,"strange for someone who's ""supposed"" to be hol...",2.134513,0.067230,satisfied


In [51]:
topic_encoded['label'].value_counts()

satisfied       152
dissatisfied      4
Name: label, dtype: int64

In [53]:
topic_encoded['text'][152]

'2 P ~— = rR a. EN a €'